In [1]:
!pip install gym
import numpy as np
import gym
import random
import time
from IPython.display import clear_output

     |████████████████████████████████| 1.6 MB 4.5 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 6.4 MB/s eta 0:00:01
  Created wheel for gym: filename=gym-0.18.0-py3-none-any.whl size=1656449 sha256=d64b7746740c3c9f8193b038c053b5b3bebd0ccd75bdda990583718c02279ace
  Stored in directory: /Users/roxanneli/Library/Caches/pip/wheels/99/f7/e3/d6f0f120ac047c1e5de2ae34930e7bf6e8de1c7a4d5fa68555
Successfully built gym
  Attempting uninstall: cloudpickle
    Found existing installation: cloudpickle 0.6.1
    Uninstalling cloudpickle-0.6.1:
      Successfully uninstalled cloudpickle-0.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 3.3.2 requires pyqt5<5.10; python_version >= "3", but you have pyqt5 5.15.2 which is incompatible.


In [2]:
env = gym.make("FrozenLake-v0")

In [4]:
action_space_size = env.action_space.n
state_space_size = env.observation_space.n

q_table = np.zeros((state_space_size, action_space_size))
print(q_table)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [5]:
num_episodes = 10000
max_steps_per_episode = 100

learning_rate = 0.1
discount_rate = 0.99

exploration_rate = 1
max_exploration_rate = 1
min_exploration_rate = 0.01
exploration_decay_rate = 0.01

In [10]:
rewards_all_episodes = []

for episode in range(num_episodes):
    
    state = env.reset()
    done = False
    rewards_current_episode = 0
    
    for step in range(max_steps_per_episode):
        
        #exploration-exploitation trade-off
        exploration_rate_threshold = random.uniform(0,1)
        if exploration_rate_threshold > exploration_rate:
            action = np.argmax(q_table[state,:])
        else:
            action = env.action_space.sample()
        
        new_state, reward, done, info = env.step(action)
        
        #update Q-table for Q(s,a)
        q_table[state,action] = (1-learning_rate)*q_table[state,action] + learning_rate*(reward+discount_rate*np.max(q_table[new_state,:]))
        state = new_state
        rewards_current_episode += reward
        
        if done==True:
            break
            
    # exploration rate decay
    exploration_rate = min_exploration_rate + (max_exploration_rate-min_exploration_rate)*np.exp(-exploration_decay_rate*episode)
    
    rewards_all_episodes.append(rewards_current_episode)
    

#calculate and print the average reward per thousand episodes   
rewards_per_thousand_episodes = np.split(np.array(rewards_all_episodes), num_episodes/1000)
count = 1000
print("**********Average reward per thousand episodes**********\n")
for r in rewards_per_thousand_episodes:
    print(count,":", str(sum(r)/1000))
    count += 1000
    

#print updated Q-table
print("\n\n**********Q-table**********\n")
print(q_table)

**********Average reward per thousand episodes**********

1000 : 0.059
2000 : 0.197
3000 : 0.412
4000 : 0.541
5000 : 0.622
6000 : 0.641
7000 : 0.662
8000 : 0.662
9000 : 0.676
10000 : 0.686


**********Q-table**********

[[0.49961964 0.49966596 0.49891309 0.49699358]
 [0.41243756 0.33998198 0.29015742 0.47070699]
 [0.42318605 0.41879501 0.41221007 0.44560371]
 [0.22790743 0.30025132 0.26206624 0.43651175]
 [0.50584533 0.33719026 0.36092228 0.31362979]
 [0.         0.         0.         0.        ]
 [0.22844987 0.16491628 0.21031669 0.14447971]
 [0.         0.         0.         0.        ]
 [0.36071996 0.4239489  0.49035483 0.51487347]
 [0.51237019 0.55554776 0.40144704 0.33136155]
 [0.49911026 0.36164205 0.35193203 0.38500267]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.4651693  0.54099952 0.66894176 0.51211247]
 [0.73507607 0.77513247 0.72935421 0.72383627]
 [0.         0.         0.         0.        ]]
